In [1]:
import os
### QAOA_realestate가 있는 폴더로 지정
dir_path = "/Users/minhyeong-gyu/Documents/GitHub/QuantumVariableSelection"
os.chdir(dir_path)
import sys
module_path = dir_path + "/Module"
if module_path not in sys.path:
    sys.path.append(module_path)

from optimizer import optimizer as opt
from optimizer import basefunctions as bf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import font_manager, rc
# 윈도우 한글 폰트
# font_path = "C:/Windows/Fonts/NGULIM.TTF"
# font = font_manager.FontProperties(fname=font_path).get_name()
# rc('font', family=font)

# Mac 한글 폰트
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

import seaborn as sns
from dwave.system import LeapHybridCQMSampler
%matplotlib inline

In [2]:
beta_coef = np.concatenate([np.random.normal(5,2,20),np.zeros(80)])
X,y = bf.generate_dependent_sample(1000,100,beta_coef,epsilon=30)
lmbd = 0.05

In [12]:
ga_qubo = opt.GeneticAlgorithm("QUBO",
                n_gen=100, 
                n_sol=40, 
                n_par=20 )
ga_qubo_result = ga_qubo.optimize(X,y,lmbd)
sa_qubo = opt.SimulatedAnnealing("QUBO")
sa_qubo_result = sa_qubo.optimize(X,y,lmbd,reps=10)

/Users/minhyeong-gyu/Documents/GitHub/QuantumVariableSelection/Module/optimizer/basefunctions.py:88: RuntimeWarning: invalid value encountered in reciprocal
  result = np.array(SSRP)**(-1)*SSRF
/Users/minhyeong-gyu/Documents/GitHub/QuantumVariableSelection/Module/optimizer/optimizer.py:91: RuntimeWarning: overflow encountered in exp
  if np.random.rand(1) <= min(1, np.exp((bf.get_QB(theta_temp,Q,-1*beta,lamda)-bf.get_QB(theta_star,Q,-1*beta,lamda))/tau)):


In [13]:
ga_aic = opt.GeneticAlgorithm(mode = "AIC",
                n_gen=100, 
                n_sol=40, 
                n_par=20)
ga_aic_result = ga_qubo.optimize(X,y,lmbd)
sa_aic = opt.SimulatedAnnealing("AIC")
sa_aic_result = sa_qubo.optimize(X,y,lmbd,reps=10)

/Users/minhyeong-gyu/Documents/GitHub/QuantumVariableSelection/Module/optimizer/basefunctions.py:88: RuntimeWarning: invalid value encountered in reciprocal
  result = np.array(SSRP)**(-1)*SSRF
/Users/minhyeong-gyu/Documents/GitHub/QuantumVariableSelection/Module/optimizer/optimizer.py:91: RuntimeWarning: overflow encountered in exp
  if np.random.rand(1) <= min(1, np.exp((bf.get_QB(theta_temp,Q,-1*beta,lamda)-bf.get_QB(theta_star,Q,-1*beta,lamda))/tau)):


In [14]:
X_sa_qubo = X[:,sa_qubo_result.astype(bool)]
X_ga_qubo = X[:,ga_qubo_result.astype(bool)]
X_sa_aic = X[:,sa_aic_result.astype(bool)]
X_ga_aic = X[:,ga_aic_result.astype(bool)]

In [18]:
AIC_list = [
    bf.get_aic(X,y),
    bf.get_aic(X_sa_aic,y),
    bf.get_aic(X_ga_aic,y),
    bf.get_aic(X_sa_qubo,y),
    bf.get_aic(X_ga_qubo,y)]

QUBO_list = [
    bf.get_QUBO(X,y,lmbd),
    bf.get_QUBO(X_sa_aic,y,lmbd),
    bf.get_QUBO(X_ga_aic,y,lmbd),
    bf.get_QUBO(X_sa_qubo,y,lmbd),
    bf.get_QUBO(X_ga_qubo,y,lmbd)]

MSPE_list = [
    bf.get_MSPE(X,y,0.8),
    bf.get_MSPE(X_sa_aic,y,0.8),
    bf.get_MSPE(X_ga_aic,y,0.8),
    bf.get_MSPE(X_sa_qubo,y,0.8),
    bf.get_MSPE(X_ga_qubo,y,0.8)]

R2_list = [
    bf.get_prediction_R2(X,y,0.8),
    bf.get_prediction_R2(X_sa_aic,y,0.8),
    bf.get_prediction_R2(X_ga_aic,y,0.8),    
    bf.get_prediction_R2(X_sa_qubo,y,0.8),
    bf.get_prediction_R2(X_ga_qubo,y,0.8)]

CN_list = [
    bf.get_CN(X),
    bf.get_CN(X_sa_aic),
    bf.get_CN(X_ga_aic),
    bf.get_CN(X_sa_qubo),
    bf.get_CN(X_ga_qubo)]


result_table = pd.DataFrame([AIC_list,QUBO_list,MSPE_list,R2_list,CN_list])
result_table.columns = ["Original","SA_AIC","GA_AIC","SA_QUBO","GA_QUBO"]
result_table.index = ["AIC_list","QUBO_list","MSPE","R2","CN"]

/Users/minhyeong-gyu/Documents/GitHub/QuantumVariableSelection/Module/optimizer/basefunctions.py:88: RuntimeWarning: invalid value encountered in reciprocal
  result = np.array(SSRP)**(-1)*SSRF


In [19]:
result_table

,Original,SA_AIC,GA_AIC,SA_QUBO,GA_QUBO
AIC_list,6895.977379,8162.261767,8372.115699,8172.906549,8297.854468
QUBO_list,5.715941,1.914223,2.370875,1.741336,2.387763
MSPE,2.388663,4.651134,4.770355,4.289099,4.331387
R2,0.627703,0.406248,0.261583,0.394170,0.268922
CN,25.695256,3.561128,4.062565,2.556361,4.318683


In [44]:
AIC_list_total =[]
QUBO_list_total = []
MSPE_list_total = []
R2_list_total = []
CN_list_total = []

for ite in range(2):
    beta_coef = np.concatenate([np.random.normal(5,2,20),np.zeros(80)])
    X,y = bf.generate_dependent_sample(1000,100,beta_coef,epsilon=30)
    lmbd = 0.05

    ga_qubo = opt.GeneticAlgorithm("QUBO")
    ga_qubo_result = ga_qubo.optimize(X,y,lmbd)
    sa_qubo = opt.SimulatedAnnealing("QUBO")
    sa_qubo_result = sa_qubo.optimize(X,y,lmbd,reps=10)
    ga_aic = opt.GeneticAlgorithm(mode = "AIC")
    ga_aic_result = ga_qubo.optimize(X,y,lmbd)
    sa_aic = opt.SimulatedAnnealing("AIC")
    sa_aic_result = sa_qubo.optimize(X,y,lmbd,reps=10)

    X_sa_qubo = X[:,sa_qubo_result.astype(bool)]
    X_ga_qubo = X[:,ga_qubo_result.astype(bool)]
    X_sa_aic = X[:,sa_aic_result.astype(bool)]
    X_ga_aic = X[:,ga_aic_result.astype(bool)]

    AIC_list = [
        bf.get_aic(X,y),
        bf.get_aic(X_sa_aic,y),
        bf.get_aic(X_ga_aic,y),
        bf.get_aic(X_sa_qubo,y),
        bf.get_aic(X_ga_qubo,y)]

    QUBO_list = [
        bf.get_QUBO(X,y,lmbd),
        bf.get_QUBO(X_sa_aic,y,lmbd),
        bf.get_QUBO(X_ga_aic,y,lmbd),
        bf.get_QUBO(X_sa_qubo,y,lmbd),
        bf.get_QUBO(X_ga_qubo,y,lmbd)]

    MSPE_list = [
        bf.get_MSPE(X,y,0.8),
        bf.get_MSPE(X_sa_aic,y,0.8),
        bf.get_MSPE(X_ga_aic,y,0.8),
        bf.get_MSPE(X_sa_qubo,y,0.8),
        bf.get_MSPE(X_ga_qubo,y,0.8)]

    R2_list = [
        bf.get_prediction_R2(X,y,0.8),
        bf.get_prediction_R2(X_sa_aic,y,0.8),
        bf.get_prediction_R2(X_ga_aic,y,0.8),    
        bf.get_prediction_R2(X_sa_qubo,y,0.8),
        bf.get_prediction_R2(X_ga_qubo,y,0.8)]

    CN_list = [
        bf.get_CN(X),
        bf.get_CN(X_sa_aic),
        bf.get_CN(X_ga_aic),
        bf.get_CN(X_sa_qubo),
        bf.get_CN(X_ga_qubo)]

    AIC_list_total += [AIC_list]
    QUBO_list_total += [QUBO_list]
    MSPE_list_total += [MSPE_list]
    R2_list_total += [R2_list]
    CN_list_total += [CN_list]

target = AIC_list_total
AIC_result = pd.DataFrame(target).apply(lambda x: str(round(np.mean(x),2)))+"("+pd.DataFrame(target).apply(lambda x: str(round(np.std(x),2)))+")"
target = QUBO_list_total
QUBO_result = pd.DataFrame(target).apply(lambda x: str(round(np.mean(x),2)))+"("+pd.DataFrame(target).apply(lambda x: str(round(np.std(x),2)))+")"
target = MSPE_list_total
MSPE_result = pd.DataFrame(target).apply(lambda x: str(round(np.mean(x),2)))+"("+pd.DataFrame(target).apply(lambda x: str(round(np.std(x),2)))+")"
target = R2_list_total
R2_result = pd.DataFrame(target).apply(lambda x: str(round(np.mean(x),2)))+"("+pd.DataFrame(target).apply(lambda x: str(round(np.std(x),2)))+")"
target = CN_list_total
CN_result = pd.DataFrame(target).apply(lambda x: str(round(np.mean(x),2)))+"("+pd.DataFrame(target).apply(lambda x: str(round(np.std(x),2)))+")"


result_table = pd.DataFrame([AIC_result,QUBO_result,MSPE_result,R2_result,CN_result])
result_table.columns = ["Original","SA_AIC","GA_AIC","SA_QUBO","GA_QUBO"]
result_table.index = ["AIC_list","QUBO_list","MSPE","R2","CN"]



/Users/minhyeong-gyu/Documents/GitHub/QuantumVariableSelection/Module/optimizer/basefunctions.py:88: RuntimeWarning: invalid value encountered in reciprocal
  result = np.array(SSRP)**(-1)*SSRF
/Users/minhyeong-gyu/Documents/GitHub/QuantumVariableSelection/Module/optimizer/optimizer.py:91: RuntimeWarning: overflow encountered in exp
  if np.random.rand(1) <= min(1, np.exp((bf.get_QB(theta_temp,Q,-1*beta,lamda)-bf.get_QB(theta_star,Q,-1*beta,lamda))/tau)):
/Users/minhyeong-gyu/Documents/GitHub/QuantumVariableSelection/Module/optimizer/optimizer.py:91: RuntimeWarning: overflow encountered in exp
  if np.random.rand(1) <= min(1, np.exp((bf.get_QB(theta_temp,Q,-1*beta,lamda)-bf.get_QB(theta_star,Q,-1*beta,lamda))/tau)):
/Users/minhyeong-gyu/Documents/GitHub/QuantumVariableSelection/Module/optimizer/basefunctions.py:88: RuntimeWarning: invalid value encountered in reciprocal
  result = np.array(SSRP)**(-1)*SSRF
/Users/minhyeong-gyu/Documents/GitHub/QuantumVariableSelection/Module/optimizer/

In [53]:
from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")

In [68]:
import random

In [80]:
random_index = str(random.randrange(0,99999)).zfill(5)

'22351'

In [45]:
result_table.to_csv("result/simulation/linear_samples"+current_time+".csv")

,Original,SA_AIC,GA_AIC,SA_QUBO,GA_QUBO
AIC_list,6872.08(14.77),7784.07(595.19),7894.9(38.8),7823.93(627.73),7846.85(30.43)
QUBO_list,5.92(0.74),3.1(1.13),3.8(0.45),3.12(1.22),3.79(0.37)
MSPE,2.24(0.07),3.68(1.01),3.74(0.09),3.68(1.24),3.75(0.17)
R2,0.71(0.03),0.56(0.1),0.5(0.03),0.52(0.11),0.52(0.02)
CN,27.29(0.97),2.66(0.59),5.41(0.28),2.74(0.19),5.26(0.13)
